In [ ]:
!# --- Required Imports ---
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float, img_as_ubyte, exposure, util, filters
from scipy import ndimage
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import cv2 # Used in example 5.6 for cv2.line

# --- Load Medical Image and Define Global Variables ---
try:
    brain_volume = data.brain()
    if brain_volume.ndim == 3:
        slice_index = brain_volume.shape[0] // 2
        image_gray_orig_c5 = brain_volume[slice_index, :, :]
    elif brain_volume.ndim == 2:
        image_gray_orig_c5 = brain_volume
        slice_index = "N/A (2D Image)"
    else:
        raise ValueError("Unexpected 'brain' image format.")
    print(f"'brain' image (slice {slice_index if brain_volume.ndim == 3 else ''}) loaded for Ch. 5.")
except Exception as e:
    print(f"Error loading 'brain': {e}. Using 'camera' as fallback.")
    image_gray_orig_c5 = data.camera()

image_float_c5 = img_as_float(image_gray_orig_c5.copy())
M_c5, N_c5 = image_float_c5.shape # Original dimensions
P_c5, Q_c5 = 2*M_c5, 2*N_c5     # Padded dimensions

# --- Helper Function for IDFT, Extraction, and Rescaling ---
def idft_process_extract(G_centralizado, P_img, Q_img, M_orig, N_orig):
    G_canto_dc = ifftshift(G_centralizado)
    img_padded_spatial = ifft2(G_canto_dc).real
    img_final = img_padded_spatial[0:M_orig, 0:N_orig]
    return exposure.rescale_intensity(img_final, out_range=(0,1))

# --- Helper Plot Function  ---
def plot_images_c5(images, titles, cmaps=None, rows=1, cols=None, figsize=(15,5)):
    num_images = len(images)
    if cols is None:
        cols = (num_images + rows - 1) // rows
    fig, axes = plt.subplots(rows, cols, figsize=figsize, squeeze=False)
    axes_flat = axes.ravel()
    if cmaps is None: cmaps_list = ['gray'] * num_images
    elif isinstance(cmaps, str): cmaps_list = [cmaps] * num_images
    else:
        cmaps_list = list(cmaps)
        if len(cmaps_list) < num_images: cmaps_list.extend(['gray']*(num_images-len(cmaps_list)))
    for i in range(len(axes_flat)):
        if i < num_images:
            img, title, cmap_val = images[i], titles[i], cmaps_list[i] # Renamed cmap to cmap_val
            axes_flat[i].imshow(img, cmap=cmap_val if img.ndim==2 else None); axes_flat[i].set_title(title); axes_flat[i].axis('off')
        else: axes_flat[i].axis('off')
    plt.tight_layout(); plt.show()


L_mov_c5 = 21
psf_movimento_c5 = np.zeros((L_mov_c5, L_mov_c5), dtype=float)
psf_movimento_c5[L_mov_c5//2, :] = 1.0
psf_movimento_c5 /= np.sum(psf_movimento_c5)

psf_movimento_padded_rolled_c5 = np.zeros((P_c5, Q_c5), dtype=float)
r_offset_c5, c_offset_c5 = L_mov_c5//2, L_mov_c5//2
for r_psf in range(L_mov_c5):
    for c_psf in range(L_mov_c5):
        idx_r_pad = (r_psf - r_offset_c5 + P_c5) % P_c5
        idx_c_pad = (c_psf - c_offset_c5 + Q_c5) % Q_c5
        psf_movimento_padded_rolled_c5[idx_r_pad, idx_c_pad] = psf_movimento_c5[r_psf, c_psf]
H_movimento_nao_centralizado_c5 = fft2(psf_movimento_padded_rolled_c5) # DC at corner

fp_para_degradacao_c5 = np.zeros((P_c5, Q_c5))
fp_para_degradacao_c5[0:M_c5, 0:N_c5] = image_float_c5
F_img_original_padded_nao_centralizado_c5 = fft2(fp_para_degradacao_c5)

G_degradada_mov_nao_centralizado_c5 = F_img_original_padded_nao_centralizado_c5 * H_movimento_nao_centralizado_c5
img_degradada_mov_padded_c5 = ifft2(G_degradada_mov_nao_centralizado_c5).real
img_degradada_mov_c5 = exposure.rescale_intensity(img_degradada_mov_padded_c5[0:M_c5, 0:N_c5], out_range=(0,1))



# --- Required Imports ---
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float, img_as_ubyte, exposure, util, filters
from scipy import ndimage
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import cv2 # Used in example 5.6 for cv2.line

# --- Load Medical Image and Define Global Variables ---
try:
    brain_volume = data.brain()
    if brain_volume.ndim == 3:
        slice_index = brain_volume.shape[0] // 2
        image_gray_orig_c5 = brain_volume[slice_index, :, :]
    elif brain_volume.ndim == 2:
        image_gray_orig_c5 = brain_volume
        slice_index = "N/A (2D Image)"
    else:
        raise ValueError("Unexpected 'brain' image format.")
    print(f"'brain' image (slice {slice_index if brain_volume.ndim == 3 else ''}) loaded for Ch. 5.")
except Exception as e:
    print(f"Error loading 'brain': {e}. Using 'camera' as fallback.")
    image_gray_orig_c5 = data.camera()

image_float_c5 = img_as_float(image_gray_orig_c5.copy())
M_c5, N_c5 = image_float_c5.shape # Original dimensions
P_c5, Q_c5 = 2*M_c5, 2*N_c5     # Padded dimensions

# --- Helper Function for IDFT, Extraction, and Rescaling ---
def idft_process_extract(G_centralizado, P_img, Q_img, M_orig, N_orig):
    G_canto_dc = ifftshift(G_centralizado)
    img_padded_spatial = ifft2(G_canto_dc).real
    img_final = img_padded_spatial[0:M_orig, 0:N_orig]
    return exposure.rescale_intensity(img_final, out_range=(0,1))

# --- Helper Plot Function (CORRIGIDA PARA USAR plot_images_c5 consistentemente) ---
def plot_images_c5(images, titles, cmaps=None, rows=1, cols=None, figsize=(15,5)):
    num_images = len(images)
    if cols is None:
        cols = (num_images + rows - 1) // rows
    fig, axes = plt.subplots(rows, cols, figsize=figsize, squeeze=False)
    axes_flat = axes.ravel()
    if cmaps is None: cmaps_list = ['gray'] * num_images
    elif isinstance(cmaps, str): cmaps_list = [cmaps] * num_images
    else:
        cmaps_list = list(cmaps)
        if len(cmaps_list) < num_images: cmaps_list.extend(['gray']*(num_images-len(cmaps_list)))
    for i in range(len(axes_flat)):
        if i < num_images:
            img, title, cmap_val = images[i], titles[i], cmaps_list[i] # Renamed cmap to cmap_val
            axes_flat[i].imshow(img, cmap=cmap_val if img.ndim==2 else None); axes_flat[i].set_title(title); axes_flat[i].axis('off')
        else: axes_flat[i].axis('off')
    plt.tight_layout(); plt.show()



# Generate the motion-degraded image and H_motion (from example 5.6)
# para que os exemplos 5.7, 5.8, 5.9 possam rodar

L_mov_c5 = 21
psf_movimento_c5 = np.zeros((L_mov_c5, L_mov_c5), dtype=float)
psf_movimento_c5[L_mov_c5//2, :] = 1.0
psf_movimento_c5 /= np.sum(psf_movimento_c5)

psf_movimento_padded_rolled_c5 = np.zeros((P_c5, Q_c5), dtype=float)
r_offset_c5, c_offset_c5 = L_mov_c5//2, L_mov_c5//2
for r_psf in range(L_mov_c5):
    for c_psf in range(L_mov_c5):
        idx_r_pad = (r_psf - r_offset_c5 + P_c5) % P_c5
        idx_c_pad = (c_psf - c_offset_c5 + Q_c5) % Q_c5
        psf_movimento_padded_rolled_c5[idx_r_pad, idx_c_pad] = psf_movimento_c5[r_psf, c_psf]
H_movimento_nao_centralizado_c5 = fft2(psf_movimento_padded_rolled_c5) # DC at corner

fp_para_degradacao_c5 = np.zeros((P_c5, Q_c5))
fp_para_degradacao_c5[0:M_c5, 0:N_c5] = image_float_c5
F_img_original_padded_nao_centralizado_c5 = fft2(fp_para_degradacao_c5)

G_degradada_mov_nao_centralizado_c5 = F_img_original_padded_nao_centralizado_c5 * H_movimento_nao_centralizado_c5
img_degradada_mov_padded_c5 = ifft2(G_degradada_mov_nao_centralizado_c5).real
img_degradada_mov_c5 = exposure.rescale_intensity(img_degradada_mov_padded_c5[0:M_c5, 0:N_c5], out_range=(0,1))


# --- Required Imports ---
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float, img_as_ubyte, transform, exposure, util, filters, morphology
from scipy import ndimage
from numpy.fft import fft2, ifft2, fftshift, ifftshift # For practice Block 4
import time
import cv2 # To draw circles in Task 1.3

# --- Helper Function to Plot Multiple Images ---
def plot_images(images, titles, cmaps=None, rows=1, cols=None, figsize=(15,5)):
    num_images = len(images)
    if cols is None:
        cols = (num_images + rows - 1) // rows

    fig, axes = plt.subplots(rows, cols, figsize=figsize, squeeze=False) # squeeze=False ensures axes is always 2D

    axes_flat = axes.ravel()

    if cmaps is None:
        cmaps_list = ['gray'] * num_images
    elif isinstance(cmaps, str):
        cmaps_list = [cmaps] * num_images
    else:
        cmaps_list = list(cmaps)
        if len(cmaps_list) < num_images:
            cmaps_list.extend(['gray'] * (num_images - len(cmaps_list)))

    for i in range(len(axes_flat)):
        if i < num_images:
            img = images[i]
            title = titles[i]
            current_cmap = cmaps_list[i]

            axes_flat[i].imshow(img, cmap=current_cmap if img.ndim == 2 else None)
            axes_flat[i].set_title(title)
            axes_flat[i].axis('off')
        else:
            axes_flat[i].axis('off') # Desliga eixos extras

    plt.tight_layout()
    plt.show()

# --- Load Medical Image ---
try:
    brain_volume = data.brain()
    if brain_volume.ndim == 3:
        slice_index = brain_volume.shape[0] // 2
        image_gray_orig = brain_volume[slice_index, :, :]
    elif brain_volume.ndim == 2:
        image_gray_orig = brain_volume
        slice_index = "N/A (2D Image)"
    else:
        raise ValueError("Unexpected 'brain' image format.")
    print(f"'brain' image (slice {slice_index if brain_volume.ndim == 3 else ''}) loaded.")
except Exception as e:
    print(f"Error loading 'brain': {e}. Using 'camera' as fallback.")
    image_gray_orig = data.camera()

# ESTAS SÃO AS VARIÁVEIS GLOBAIS PARA A PRÁTICA
image_float_g = img_as_float(image_gray_orig.copy())
image_ubyte_g = img_as_ubyte(image_gray_orig.copy())
M_g, N_g = image_float_g.shape # Global dimensions for the practice



Edge Detection with Sobel (Practice)


Objective: Apply the Sobel operator to detect edges in a medical image.


How the Code Works:

1. Load Image: Uses `image_float_g` (brain image).
2. Sobel Filter:
   * `filters.sobel_h(image_float_g)`: Computes the horizontal component (related to vertical edges).
   * `filters.sobel_v(image_float_g)`: Computes the vertical component (related to horizontal edges).
   * `filters.sobel(image_float_g)`: Computes gradient magnitude.
3. Thresholding: Applies a threshold to Sobel magnitude to produce a binary edge image.


In [ ]:
# Python Example: Edge Detection with Sobel
print("\n--- Practical Example: Edge Detection with Sobel ---")
from skimage import filters # filters was already imported, but kept here for clarity

# Apply Sobel
sob_h = filters.sobel_h(image_float_g) # Horizontal component g_x
sob_v = filters.sobel_v(image_float_g) # Vertical component g_y
mag_sobel = filters.sobel(image_float_g) # Gradient magnitude

# Threshold the magnitude to obtain a binary edge image
limiar_sobel_val = 0.05 # Adjust this value
bordas_sobel = mag_sobel > limiar_sobel_val

plot_images_c5([image_float_g, sob_h, sob_v, mag_sobel, bordas_sobel],
               ["Original", "Sobel Gx", "Sobel Gy", "Sobel Magnitude", f"Sobel Edges (T={limiar_sobel_val})"],
               rows=2, cols=3, figsize=(15,10),
               cmaps=['gray','gray','gray','viridis','gray']) # Use viridis for magnitude



Interpreting the Results (Sobel):

* Original: Input image.
* Sobel Gx: Highlights vertical edges.
* Sobel Gy: Highlights horizontal edges.
* Sobel Magnitude: Overall edge strength map.
* Thresholded Edges: Binary edge image after thresholding.


Short Exercise (Edge Detection):

* Change `limiar_sobel_val` to `0.02`, `0.1`, and `0.2`. What happens to the number of detected edges? How does sensitivity change?


Edge Detection with Canny (Practice)

Objective: Apply Canny edge detection and observe the effect of its parameters.


How the Code Works:

1. `feature.canny(image_float_g, sigma=..., low_threshold=..., high_threshold=...)`
   * `image_float_g`: input image.
   * `sigma`: Gaussian smoothing before gradient computation.
   * `low_threshold`, `high_threshold`: hysteresis thresholds for edge linking.
2. Visualization: Displays original image and Canny output.


In [ ]:
# Python Example: Edge Detection with Canny - FIXED
print("\n--- Practical Example: Edge Detection with Canny ---")
from skimage import feature # CORRECT import for Canny

# Canny parameters (these may need tuning for your image)
sigma_canny_c10 = 1.5 # Slightly increase sigma for more initial smoothing
low_thresh_canny_c10 = 0.1
high_thresh_canny_c10 = 0.25


bordas_canny_c10 = feature.canny(image_float_g, sigma=sigma_canny_c10,
                                 low_threshold=low_thresh_canny_c10,
                                 high_threshold=high_thresh_canny_c10)

plot_images_c5([image_float_g, bordas_canny_c10],
               ["Original", f"Canny Edges (s={sigma_canny_c10}, L={low_thresh_canny_c10:.2f}, H={high_thresh_canny_c10:.2f})"],
               cmaps=['gray','gray']) # plot_images_c5 is the corrected function

Interpreting the Results (Canny):

* Original: Input image.
* Canny Edges: Binary image with edges detected by Canny.
* In general, Canny tends to produce thinner and better-connected edges than simple gradient thresholding.


Exercise (Canny):

1. Vary `sigma_canny_c10` (e.g., 0.5, 2.0, 3.0). How does this affect detected edges? What happens to noise and fine details?
2. Adjust `low_thresh_canny_c10` and `high_thresh_canny_c10`. How do false positives and missed edges change?


Python - Simple Global Thresholding and Histogram (Practice)


Objective: Visualize the image histogram and apply a manually chosen global threshold.


How the Code Works:

1. Histogram: `exposure.histogram(image_ubyte_g)` computes histogram of the 8-bit image.
2. Manual Threshold: choose `limiar_manual_ubyte` and convert to float range when needed.
3. Segmentation: creates a binary image from threshold comparison.


In [ ]:
# Python Example: Simple Global Thresholding and Histogram
print("\n--- Practical Example: Simple Global Thresholding ---")

# Compute and plot the ubyte image histogram
hist_original_c10, bins_orig_c10 = exposure.histogram(image_ubyte_g, nbins=256, source_range='image')

# Choose a manual threshold by inspecting the histogram (value between 0-255)
limiar_manual_ubyte = 80 # Example, adjust this value!
# Convert to float image scale [0,1] if needed
limiar_manual_float = limiar_manual_ubyte / 255.0

# Apply threshold to float image
img_limiarizada_manual = image_float_g > limiar_manual_float

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].imshow(image_float_g, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')

axes[1].plot(bins_orig_c10, hist_original_c10, color='blue')
axes[1].axvline(limiar_manual_ubyte, color='red', linestyle='dashed', linewidth=2)
axes[1].set_title(f'Histogram and Manual Threshold ({limiar_manual_ubyte})')
axes[1].set_xlabel('Gray Level'); axes[1].set_ylabel('Pixel Count')

axes[2].imshow(img_limiarizada_manual, cmap='gray'); axes[2].set_title(f'Segmented (T={limiar_manual_ubyte})'); axes[2].axis('off')
plt.show()

Exercise (Manual Thresholding):

1. Observe the histogram of `image_ubyte_g` and identify a possible valley between peaks.
2. Change `limiar_manual_ubyte` and compare segmentation quality.


Python - Otsu Method and Adaptive Thresholding (Practice)


Objective: Apply Otsu's method for automatic global thresholding and an adaptive thresholding method.


How the Code Works:

1. Otsu Method:
   * `filters.threshold_otsu(image_ubyte_g)`: automatically computes an optimal global threshold.
   * `img_otsu = image_ubyte_g > limiar_otsu`.
2. Adaptive Thresholding:
   * `filters.threshold_local(...)` computes a local threshold per pixel neighborhood.
   * Parameters `block_size_adapt` and `offset_adapt` control locality and bias.


In [ ]:
# Python Example: Otsu Method and Adaptive Thresholding
print("\n--- Practical Example: Otsu and Adaptive Thresholding ---")

# Otsu Method (Global)
limiar_otsu = filters.threshold_otsu(image_ubyte_g) # Computes optimal threshold
img_otsu = image_ubyte_g > limiar_otsu          # Applies threshold

# Local Adaptive Thresholding (e.g., using Gaussian neighborhood mean)
# block_size must be odd and greater than 1
block_size_adapt = 35 # Local neighborhood size. Adjust this value.
offset_adapt = 0.02   # Offset to adjust local threshold. Adjust this value.
# threshold_local returns a threshold image, one value per pixel.
limiares_adaptativos = filters.threshold_local(image_float_g, block_size=block_size_adapt,
                                             method='gaussian', offset=offset_adapt)
img_adaptativa = image_float_g > limiares_adaptativos

plot_images_c5([image_float_g, img_otsu, img_adaptativa],
               ["Original", f"Otsu (T={limiar_otsu})", f"Adaptive (block={block_size_adapt}, offs={offset_adapt})"],
               cmaps=['gray','gray','gray'])

Interpreting the Results (Otsu and Adaptive):

1. Otsu: global separation based on histogram distribution.
2. Adaptive: better handles nonuniform illumination and local contrast changes.


Exercise (Otsu and Adaptive):


1. For adaptive thresholding, try different `block_size_adapt` (e.g., 15, 55, 101) and `offset_adapt` (e.g., -0.05, 0, 0.05). How do these parameters affect segmentation?
2. Compare Otsu vs adaptive on this image. Which one is better and why?


Python - Region Growing (Conceptual/Simplified)


Objective: Illustrate the basic concept of seed-based region growing. A full implementation can be more complex, but this simulates the core idea.


How the Code Works:

1. Seed and Similarity Threshold:
   * `idx_linha_semente`, `idx_col_semente`: initial seed pixel.
   * `limiar_similaridade_rg`: maximum intensity difference to include neighboring pixels.
2. Queue-based expansion:
   * Starts from seed and checks 8-connected neighbors.
   * Adds neighbors that satisfy similarity criterion and are not visited.
3. Output:
   * Binary grown region mask and original image with seed marker.


In [ ]:
# Python Example: Region Growing (Simplified)
print("\n--- Practical Example: Region Growing (Simplified) ---")
from collections import deque # To use a queue

# Use image_float_g and its dimensions M_g, N_g

# Parameters for region growing
# Choose a seed. For the brain image, a bright interior point works well.
# Inspect image_float_g to choose a good seed.
# Example (may need adjustment):
idx_linha_semente = M_g // 2
idx_col_semente = N_g // 2
# Check if the initial seed is not too dark (background)
if image_float_g[idx_linha_semente, idx_col_semente] < 0.1: # If it is too dark, try another
    idx_linha_semente = M_g // 3
    idx_col_semente = N_g // 2
    if image_float_g[idx_linha_semente, idx_col_semente] < 0.1:
         idx_linha_semente = M_g // 2 + 20; idx_col_semente = N_g //2 + 20


valor_semente = image_float_g[idx_linha_semente, idx_col_semente]
print(f"Seed at ({idx_linha_semente},{idx_col_semente}) with value {valor_semente:.2f}")

limiar_similaridade_rg = 0.1 # Maximum intensity difference to be considered similar

img_segmentada_rg = np.zeros_like(image_float_g, dtype=np.uint8)
fila_rg = deque()

if 0 <= valor_semente <= 1: # Ensure seed is valid
    img_segmentada_rg[idx_linha_semente, idx_col_semente] = 1
    fila_rg.append((idx_linha_semente, idx_col_semente))

    # Process the queue
    while fila_rg:
        r_atual, c_atual = fila_rg.popleft()

        # Check 8 neighbors
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue # Skip the current pixel itself

                r_viz, c_viz = r_atual + dr, c_atual + dc

                # Check image boundaries
                if 0 <= r_viz < M_g and 0 <= c_viz < N_g:
                    # Check if not visited and satisfies similarity criterion
                    if img_segmentada_rg[r_viz, c_viz] == 0 and \
                       np.abs(image_float_g[r_viz, c_viz] - valor_semente) < limiar_similaridade_rg:
                        # (Simple criterion: similarity to original seed)
                        # (A more robust criterion would use the mean of the grown region)
                        img_segmentada_rg[r_viz, c_viz] = 1
                        fila_rg.append((r_viz, c_viz))
else:
    print("Invalid seed value or outside expected range.")


# Visualization
img_original_com_semente = img_as_ubyte(image_float_g.copy())
img_original_com_semente = cv2.cvtColor(img_original_com_semente, cv2.COLOR_GRAY2BGR)
if 0 <= valor_semente <= 1: # Draw only if seed is valid
    cv2.circle(img_original_com_semente, (idx_col_semente, idx_linha_semente), 5, (0,255,0), -1) # Seed at verde


plot_images_c5([img_original_com_semente, img_segmentada_rg],
               ["Original with Seed", f"Grown Region (Seed at ({idx_linha_semente},{idx_col_semente}), T={limiar_similaridade_rg})"],
               cmaps=['gray','gray'], figsize=(10,5)) # cmap does not apply to img_original_com_semente if BGR

Interpreting the Results (Region Growing):

* Original with Seed: shows where growth starts.
* Grown Region: shows connected pixels similar to seed under chosen threshold.
* Behavior depends strongly on seed location and similarity threshold.


Exercise (Region Growing):

1. Change `idx_linha_semente` and `idx_col_semente` to different image areas.
2. Change `limiar_similaridade_rg` (e.g., 0.05, 0.15, 0.25).
3. Observe under-segmentation vs over-segmentation behavior.
